# test data - tourism data

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os
from datetime import datetime, timedelta
import pandas as pd
import math
import numpy as np
import random
from tqdm import trange

from io import BytesIO
from urllib.request import urlopen
from zipfile import ZipFile

from math import sqrt
from scipy import stats
from collections import Counter

In [2]:
window_size = 8
stride_size = 2

train_start = '1998-01-01'
train_end = '2014-10-01'
test_start = '2014-01-01' #need additional 7 days as given info 
test_end = '2016-10-01'

In [3]:
data = pd.read_csv(f'data/tourism.csv',index_col=0)  # 输入数据格式 sku-group time value
group_names = ['State','Region','Purpose']
time_name = 'Quarter'
target_name = 'Trips'

data['sku']=data[group_names].apply(lambda x: 'total'+'_'+'_'.join(x),axis=1)

hier_dict={}
hier_dict['total']=[]
for group_idx in range(len(group_names)-1):
    group = group_names[group_idx]
    for k in data[group].unique():
        temp_list = list(data.loc[data[group]==k,group_names[group_idx+1]].unique())
        if group_idx==0:
            hier_dict['total_'+k] = ['total_'+k+"_"+t for t in temp_list]
            hier_dict['total'].append('total_'+k)
        else:
            for k2 in sum(list(hier_dict.values())[-1::-1], []):
                if (k2.find(k)!=-1)&(len(k2.split('_'))==group_idx+2):
                    hier_dict[k2] = [k2+"_"+t for t in temp_list]

In [4]:
data = pd.pivot_table(data,values=target_name,index=[time_name],columns=['sku'])
data.index=pd.to_datetime(data.index)
data = data.sort_index()


In [28]:
data.index=pd.to_datetime(data.index)
data = data.asfreq('QS')

In [32]:
data.index = pd.PeriodIndex(data.index,freq="Q")

In [35]:
import datetime

In [43]:
data['total_ACT_Canberra_Business'][-8:]

Quarter
2015Q1    102.019685
2015Q2    151.683413
2015Q3    214.958120
2015Q4    187.314659
2016Q1    153.307540
2016Q2    195.719394
2016Q3    219.421298
2016Q4    186.399072
Freq: Q-DEC, Name: total_ACT_Canberra_Business, dtype: float64

In [39]:
datetime.datetime.strftime(data.index.to_timestamp(), '%Y%m%d')

TypeError: descriptor 'strftime' for 'datetime.date' objects doesn't apply to a 'DatetimeIndex' object

In [38]:
list(data.index.to_timestamp())

[Timestamp('1998-01-01 00:00:00', freq='QS-OCT'),
 Timestamp('1998-04-01 00:00:00', freq='QS-OCT'),
 Timestamp('1998-07-01 00:00:00', freq='QS-OCT'),
 Timestamp('1998-10-01 00:00:00', freq='QS-OCT'),
 Timestamp('1999-01-01 00:00:00', freq='QS-OCT'),
 Timestamp('1999-04-01 00:00:00', freq='QS-OCT'),
 Timestamp('1999-07-01 00:00:00', freq='QS-OCT'),
 Timestamp('1999-10-01 00:00:00', freq='QS-OCT'),
 Timestamp('2000-01-01 00:00:00', freq='QS-OCT'),
 Timestamp('2000-04-01 00:00:00', freq='QS-OCT'),
 Timestamp('2000-07-01 00:00:00', freq='QS-OCT'),
 Timestamp('2000-10-01 00:00:00', freq='QS-OCT'),
 Timestamp('2001-01-01 00:00:00', freq='QS-OCT'),
 Timestamp('2001-04-01 00:00:00', freq='QS-OCT'),
 Timestamp('2001-07-01 00:00:00', freq='QS-OCT'),
 Timestamp('2001-10-01 00:00:00', freq='QS-OCT'),
 Timestamp('2002-01-01 00:00:00', freq='QS-OCT'),
 Timestamp('2002-04-01 00:00:00', freq='QS-OCT'),
 Timestamp('2002-07-01 00:00:00', freq='QS-OCT'),
 Timestamp('2002-10-01 00:00:00', freq='QS-OCT'),


In [5]:
for k in list(hier_dict.keys())[-1::-1]:
    data[k] = data[hier_dict[k]].sum(axis=1)

In [6]:
def gen_covariates(times, num_covariates):
    covariates = np.zeros((times.shape[0], num_covariates))
    for i, input_time in enumerate(times):
        covariates[i, 1] = input_time.quarter
        covariates[i, 2] = input_time.year
    for i in range(1,num_covariates):
        covariates[:,i] = stats.zscore(covariates[:,i])
    return covariates[:, :num_covariates]
num_covariates = 3
covariates = gen_covariates(data[train_start:test_end].index, num_covariates)

In [7]:
train_data = data[train_start:train_end].values
test_data = data[test_start:test_end].values
# data_start = (train_data!=0).argmax(axis=0) #find first nonzero value in each time series
total_time = data.shape[0] #76
num_series = data.shape[1] #85

In [14]:
data_start

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 3, 0,
       0, 0, 0, 0, 0, 0, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       5, 0, 2, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 2, 0, 0, 0, 4, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [17]:
import json

In [19]:
with open('experiments/base_model/params.json','r+') as f:
    params=json.load(f)

In [20]:
params

{'learning_rate': 0.001,
 'batch_size': 85,
 'lstm_layers': 3,
 'num_epochs': 20,
 'train_window': 8,
 'test_window': 8,
 'predict_start': 4,
 'test_predict_start': 4,
 'predict_steps': 4,
 'num_class': 85,
 'cov_dim': 3,
 'lstm_hidden_dim': 40,
 'embedding_dim': 20,
 'sample_times': 2000,
 'lstm_dropout': 0.1,
 'predict_batch': 85}

In [8]:
data = train_data.copy()
train = True
time_len = data.shape[0]
#print("time_len: ", time_len)
input_size = window_size-stride_size
windows_per_series = np.full((num_series), (time_len-input_size) // stride_size)
#print("windows pre: ", windows_per_series.shape)
if train: windows_per_series -= (data_start+stride_size-1) // stride_size 
#print("data_start: ", data_start.shape)
#print(data_start)
#print("windows: ", windows_per_series.shape)
#print(windows_per_series)
total_windows = np.sum(windows_per_series)
x_input = np.zeros((total_windows, window_size, 1 + num_covariates + 1), dtype='float32')
label = np.zeros((total_windows, window_size), dtype='float32')
v_input = np.zeros((total_windows, 2), dtype='float32')
#cov = 3: ground truth + age + day_of_week + hour_of_day + num_series
#cov = 4: ground truth + age + day_of_week + hour_of_day + month_of_year + num_series
count = 0
if not train:
    covariates = covariates[-time_len:]
for series in trange(num_series):
    cov_age = stats.zscore(np.arange(total_time-data_start[series]))
    if train:
        covariates[data_start[series]:time_len, 0] = cov_age[:time_len-data_start[series]]
    else:
        covariates[:, 0] = cov_age[-time_len:]
    for i in range(windows_per_series[series]):
        if train:
            window_start = stride_size*i+data_start[series]
        else:
            window_start = stride_size*i
        window_end = window_start+window_size
        '''
        print("x: ", x_input[count, 1:, 0].shape)
        print("window start: ", window_start)
        print("window end: ", window_end)
        print("data: ", data.shape)
        print("d: ", data[window_start:window_end-1, series].shape)
        '''
        x_input[count, 1:, 0] = data[window_start:window_end-1, series] # 这里少一个是因为z_t-1，但为什么不对齐，为什么不是x_input[count, :-1, 0]
        x_input[count, :, 1:1+num_covariates] = covariates[window_start:window_end, :]
        x_input[count, :, -1] = series # idx
        label[count, :] = data[window_start:window_end, series] # label是这一段序列的值
        nonzero_sum = (x_input[count, 1:input_size, 0]!=0).sum()
        if nonzero_sum == 0:
            v_input[count, 0] = 0 # 这个v好像是论文里面 scale handling部分
        else:
            v_input[count, 0] = np.true_divide(x_input[count, 1:input_size, 0].sum(),nonzero_sum)+1
            x_input[count, :, 0] = x_input[count, :, 0]/v_input[count, 0]
            if train:
                label[count, :] = label[count, :]/v_input[count, 0]
        count += 1

100%|██████████| 393/393 [00:01<00:00, 233.72it/s]


In [9]:
x_input.shape

(12156, 8, 5)

In [10]:
12156/8

1519.5

In [ ]:

train_x_input, train_v_input, train_label = prep_data(train_data, covariates, data_start)

In [54]:
data.columns

Index(['total_ACT_Canberra_Business', 'total_ACT_Canberra_Holiday',
       'total_ACT_Canberra_Other', 'total_ACT_Canberra_Visiting',
       'total_New South Wales_Blue Mountains_Business',
       'total_New South Wales_Blue Mountains_Holiday',
       'total_New South Wales_Blue Mountains_Other',
       'total_New South Wales_Blue Mountains_Visiting',
       'total_New South Wales_Capital Country_Business',
       'total_New South Wales_Capital Country_Holiday',
       ...
       'total_South Australia_Adelaide Hills',
       'total_South Australia_Adelaide', 'total_ACT', 'total_Queensland',
       'total_New South Wales', 'total_Victoria', 'total_Western Australia',
       'total_Northern Territory', 'total_South Australia', 'total'],
      dtype='object', name='sku', length=393)

In [3]:
tourism_data = pd.read_csv('data/tourism.csv',index_col=0)
tourism_data = pd.DataFrame(tourism_data.groupby(['Quarter','Region','State'],group_keys=False).sum('Trips')).reset_index()
tourism_data['area']=tourism_data['State']+'_'+tourism_data['Region']

In [12]:
tourism_data

,Quarter,Region,State,Trips,area
0,1998/1/1,Adelaide,South Australia,658.553895,South Australia_Adelaide
1,1998/1/1,Adelaide Hills,South Australia,9.798630,South Australia_Adelaide Hills
2,1998/1/1,Alice Springs,Northern Territory,20.207638,Northern Territory_Alice Springs
3,1998/1/1,Australia's Coral Coast,Western Australia,132.516409,Western Australia_Australia's Coral Coast
4,1998/1/1,Australia's Golden Outback,Western Australia,161.726948,Western Australia_Australia's Golden Outback
...,...,...,...,...,...
5847,2016/7/1,Western Grampians,Victoria,70.246932,Victoria_Western Grampians
5848,2016/7/1,Whitsundays,Queensland,165.868169,Queensland_Whitsundays
5849,2016/7/1,Wilderness West,Northern Territory,21.969584,Northern Territory_Wilderness West
5850,2016/7/1,Wimmera,Victoria,24.436255,Victoria_Wimmera


In [62]:
df = pd.DataFrame(index=list(set(tourism_data.set_index(['State','Region']).index)))

In [65]:
['State','Region'][:-1]

['State']

In [4]:
hier_dict=dict.fromkeys(tourism_data['State'].unique())
for k in hier_dict.keys():
    temp_list = list(tourism_data.loc[tourism_data['State']==k,'Region'].unique())
    hier_dict[k] = [k+"_"+t for t in temp_list]

In [5]:
data = pd.pivot_table(tourism_data,values='Trips',index=['Quarter'],columns=['area'])
data.index=pd.to_datetime(data.index)
data = data.sort_index()

In [6]:
data

area,ACT_Canberra,New South Wales_Blue Mountains,New South Wales_Capital Country,New South Wales_Central Coast,New South Wales_Central NSW,New South Wales_Hunter,New South Wales_New England North West,New South Wales_North Coast NSW,New South Wales_Outback NSW,New South Wales_Riverina,...,Victoria_Spa Country,Victoria_Upper Yarra,Victoria_Western,Victoria_Western Grampians,Victoria_Wimmera,Western Australia_Australia's Coral Coast,Western Australia_Australia's Golden Outback,Western Australia_Australia's North West,Western Australia_Australia's South West,Western Australia_Experience Perth
Quarter,,,,,,,,,,,,,,,,,,,,,
1998-01-01,551.001921,195.543180,261.808142,455.166849,424.395446,674.978457,329.838506,1342.301230,96.602651,215.268912,...,86.196006,102.791022,739.350975,86.996591,18.804743,132.516409,161.726948,120.775450,474.858729,751.211958
1998-04-01,416.025623,200.408671,238.600029,343.476758,557.011571,791.887234,328.366005,1161.108602,120.648446,336.094921,...,74.356890,74.855136,359.626051,84.939977,52.482311,172.615378,164.973780,158.404387,411.622281,668.710707
1998-07-01,436.029011,253.359689,184.423752,303.670875,555.527970,566.040599,324.859168,839.368066,110.073779,228.065371,...,60.778546,59.465405,342.321758,79.974884,35.657551,173.904335,206.879934,184.619035,360.039657,662.850731
1998-10-01,449.798445,245.573846,229.088938,331.817060,590.158435,591.568947,279.796195,1183.859971,185.470023,201.123526,...,46.013284,35.238855,445.968992,116.235617,27.204455,207.002571,198.509591,138.878263,462.620050,832.706514
1999-01-01,378.572817,290.484124,252.281951,466.815571,480.129260,785.102775,327.491877,1306.877508,78.363807,200.822443,...,65.299169,67.823457,607.808299,101.765635,50.219851,198.856638,140.213443,103.337122,562.974629,830.305740
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2015-10-01,542.769655,203.906222,369.513080,364.288154,512.524298,821.778303,333.112869,1213.745435,95.876016,208.726401,...,86.963114,73.761141,531.783733,55.707346,40.190952,217.657957,211.428114,367.652262,647.774842,937.578580
2016-01-01,626.640631,239.080990,303.226820,412.911330,503.693601,858.614110,315.795461,1624.141691,60.299009,243.643135,...,95.839407,38.611450,723.103235,100.075778,46.144294,274.923899,261.694135,207.329007,967.389399,1086.068326
2016-04-01,592.608499,303.558354,322.504718,250.955972,461.043217,829.977102,392.017225,1153.697747,134.831251,279.953218,...,82.168945,63.697508,401.272195,49.527482,24.402157,279.911249,288.976805,289.814315,650.119716,960.572196


In [7]:
for k in hier_dict.keys():
    data[k] = data[hier_dict[k]].sum(axis=1)

In [8]:
data

area,ACT_Canberra,New South Wales_Blue Mountains,New South Wales_Capital Country,New South Wales_Central Coast,New South Wales_Central NSW,New South Wales_Hunter,New South Wales_New England North West,New South Wales_North Coast NSW,New South Wales_Outback NSW,New South Wales_Riverina,...,Western Australia_Australia's North West,Western Australia_Australia's South West,Western Australia_Experience Perth,South Australia,Northern Territory,Western Australia,Victoria,New South Wales,Queensland,ACT
Quarter,,,,,,,,,,,,,,,,,,,,,
1998-01-01,551.001921,195.543180,261.808142,455.166849,424.395446,674.978457,329.838506,1342.301230,96.602651,215.268912,...,120.775450,474.858729,751.211958,1732.743704,1914.709878,1641.089495,6010.424491,8039.794795,4042.779858,551.001921
1998-04-01,416.025623,200.408671,238.600029,343.476758,557.011571,791.887234,328.366005,1161.108602,120.648446,336.094921,...,158.404387,411.622281,668.710707,1394.638319,1541.405790,1576.326534,4795.246755,7166.013805,3967.069713,416.025623
1998-07-01,436.029011,253.359689,184.423752,303.670875,555.527970,566.040599,324.859168,839.368066,110.073779,228.065371,...,184.619035,360.039657,662.850731,1213.330723,1321.819326,1588.293692,4316.845170,6747.935790,4598.548477,436.029011
1998-10-01,449.798445,245.573846,229.088938,331.817060,590.158435,591.568947,279.796195,1183.859971,185.470023,201.123526,...,138.878263,462.620050,832.706514,1449.354514,1439.406267,1839.716990,4674.829118,7282.082371,4207.772812,449.798445
1999-01-01,378.572817,290.484124,252.281951,466.815571,480.129260,785.102775,327.491877,1306.877508,78.363807,200.822443,...,103.337122,562.974629,830.305740,1535.459593,1830.389536,1835.687573,5304.334195,7584.776839,4332.490850,378.572817
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2015-10-01,542.769655,203.906222,369.513080,364.288154,512.524298,821.778303,333.112869,1213.745435,95.876016,208.726401,...,367.652262,647.774842,937.578580,1695.622430,2087.124892,2382.091755,5817.562633,7865.399385,5368.858427,542.769655
2016-01-01,626.640631,239.080990,303.226820,412.911330,503.693601,858.614110,315.795461,1624.141691,60.299009,243.643135,...,207.329007,967.389399,1086.068326,1874.771647,2114.266717,2797.404767,6576.363923,8407.366451,5095.546237,626.640631
2016-04-01,592.608499,303.558354,322.504718,250.955972,461.043217,829.977102,392.017225,1153.697747,134.831251,279.953218,...,289.814315,650.119716,960.572196,1661.079180,1628.593948,2469.394281,5335.229737,7660.579621,5444.312492,592.608499


In [9]:
temp_total = data.sum(axis=1)
data['total'] = temp_total

In [10]:
temp_cols = list(data.columns)
final_cols = ['total'+'_'+c if c != 'total' else 'total' for c in temp_cols ]
data.columns = final_cols

In [11]:
data

,total_ACT_Canberra,total_New South Wales_Blue Mountains,total_New South Wales_Capital Country,total_New South Wales_Central Coast,total_New South Wales_Central NSW,total_New South Wales_Hunter,total_New South Wales_New England North West,total_New South Wales_North Coast NSW,total_New South Wales_Outback NSW,total_New South Wales_Riverina,...,total_Western Australia_Australia's South West,total_Western Australia_Experience Perth,total_South Australia,total_Northern Territory,total_Western Australia,total_Victoria,total_New South Wales,total_Queensland,total_ACT,total
Quarter,,,,,,,,,,,,,,,,,,,,,
1998-01-01,551.001921,195.543180,261.808142,455.166849,424.395446,674.978457,329.838506,1342.301230,96.602651,215.268912,...,474.858729,751.211958,1732.743704,1914.709878,1641.089495,6010.424491,8039.794795,4042.779858,551.001921,47865.088282
1998-04-01,416.025623,200.408671,238.600029,343.476758,557.011571,791.887234,328.366005,1161.108602,120.648446,336.094921,...,411.622281,668.710707,1394.638319,1541.405790,1576.326534,4795.246755,7166.013805,3967.069713,416.025623,41713.453080
1998-07-01,436.029011,253.359689,184.423752,303.670875,555.527970,566.040599,324.859168,839.368066,110.073779,228.065371,...,360.039657,662.850731,1213.330723,1321.819326,1588.293692,4316.845170,6747.935790,4598.548477,436.029011,40445.604377
1998-10-01,449.798445,245.573846,229.088938,331.817060,590.158435,591.568947,279.796195,1183.859971,185.470023,201.123526,...,462.620050,832.706514,1449.354514,1439.406267,1839.716990,4674.829118,7282.082371,4207.772812,449.798445,42685.921034
1999-01-01,378.572817,290.484124,252.281951,466.815571,480.129260,785.102775,327.491877,1306.877508,78.363807,200.822443,...,562.974629,830.305740,1535.459593,1830.389536,1835.687573,5304.334195,7584.776839,4332.490850,378.572817,45603.422805
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2015-10-01,542.769655,203.906222,369.513080,364.288154,512.524298,821.778303,333.112869,1213.745435,95.876016,208.726401,...,647.774842,937.578580,1695.622430,2087.124892,2382.091755,5817.562633,7865.399385,5368.858427,542.769655,51518.858354
2016-01-01,626.640631,239.080990,303.226820,412.911330,503.693601,858.614110,315.795461,1624.141691,60.299009,243.643135,...,967.389399,1086.068326,1874.771647,2114.266717,2797.404767,6576.363923,8407.366451,5095.546237,626.640631,54984.720748
2016-04-01,592.608499,303.558354,322.504718,250.955972,461.043217,829.977102,392.017225,1153.697747,134.831251,279.953218,...,650.119716,960.572196,1661.079180,1628.593948,2469.394281,5335.229737,7660.579621,5444.312492,592.608499,49583.595515


In [12]:
def gen_covariates(times, num_covariates):
    covariates = np.zeros((times.shape[0], num_covariates))
    for i, input_time in enumerate(times):
        covariates[i, 1] = input_time.quarter
        covariates[i, 2] = input_time.year
    for i in range(1,num_covariates):
        covariates[:,i] = stats.zscore(covariates[:,i])
    return covariates[:, :num_covariates]

In [13]:
window_size = 8
stride_size = 2
num_covariates = 3
train_start = '1998-01-01'
train_end = '2014-10-01'
test_start = '2014-01-01' #need additional 7 days as given info 
test_end = '2016-10-01'

In [14]:
covariates = gen_covariates(data[train_start:test_end].index, num_covariates)

In [19]:
data[train_start:test_end].index

DatetimeIndex(['1998-01-01', '1998-04-01', '1998-07-01', '1998-10-01',
               '1999-01-01', '1999-04-01', '1999-07-01', '1999-10-01',
               '2000-01-01', '2000-04-01', '2000-07-01', '2000-10-01',
               '2001-01-01', '2001-04-01', '2001-07-01', '2001-10-01',
               '2002-01-01', '2002-04-01', '2002-07-01', '2002-10-01',
               '2003-01-01', '2003-04-01', '2003-07-01', '2003-10-01',
               '2004-01-01', '2004-04-01', '2004-07-01', '2004-10-01',
               '2005-01-01', '2005-04-01', '2005-07-01', '2005-10-01',
               '2006-01-01', '2006-04-01', '2006-07-01', '2006-10-01',
               '2007-01-01', '2007-04-01', '2007-07-01', '2007-10-01',
               '2008-01-01', '2008-04-01', '2008-07-01', '2008-10-01',
               '2009-01-01', '2009-04-01', '2009-07-01', '2009-10-01',
               '2010-01-01', '2010-04-01', '2010-07-01', '2010-10-01',
               '2011-01-01', '2011-04-01', '2011-07-01', '2011-10-01',
      

In [15]:
train_data = data[train_start:train_end].values
test_data = data[test_start:test_end].values
data_start = (train_data!=0).argmax(axis=0) #find first nonzero value in each time series
total_time = data.shape[0] #76
num_series = data.shape[1] #85

In [8]:
def prep_data(data, covariates, data_start, train = True):
    #print("train: ", train)
    time_len = data.shape[0]
    #print("time_len: ", time_len)
    input_size = window_size-stride_size
    windows_per_series = np.full((num_series), (time_len-input_size) // stride_size)
    #print("windows pre: ", windows_per_series.shape)
    if train: windows_per_series -= (data_start+stride_size-1) // stride_size 
    #print("data_start: ", data_start.shape)
    #print(data_start)
    #print("windows: ", windows_per_series.shape)
    #print(windows_per_series)
    total_windows = np.sum(windows_per_series)
    x_input = np.zeros((total_windows, window_size, 1 + num_covariates + 1), dtype='float32')
    label = np.zeros((total_windows, window_size), dtype='float32')
    v_input = np.zeros((total_windows, 2), dtype='float32')
    #cov = 3: ground truth + age + day_of_week + hour_of_day + num_series
    #cov = 4: ground truth + age + day_of_week + hour_of_day + month_of_year + num_series
    count = 0
    if not train:
        covariates = covariates[-time_len:]
    for series in trange(num_series):
        cov_age = stats.zscore(np.arange(total_time-data_start[series]))
        if train:
            covariates[data_start[series]:time_len, 0] = cov_age[:time_len-data_start[series]]
        else:
            covariates[:, 0] = cov_age[-time_len:]
        for i in range(windows_per_series[series]):
            if train:
                window_start = stride_size*i+data_start[series]
            else:
                window_start = stride_size*i
            window_end = window_start+window_size
            '''
            print("x: ", x_input[count, 1:, 0].shape)
            print("window start: ", window_start)
            print("window end: ", window_end)
            print("data: ", data.shape)
            print("d: ", data[window_start:window_end-1, series].shape)
            '''
            x_input[count, 1:, 0] = data[window_start:window_end-1, series] # 这里少一个是因为z_t-1，但为什么不对齐，为什么不是x_input[count, :-1, 0]
            x_input[count, :, 1:1+num_covariates] = covariates[window_start:window_end, :]
            x_input[count, :, -1] = series # idx
            label[count, :] = data[window_start:window_end, series] # label是这一段序列的值
            nonzero_sum = (x_input[count, 1:input_size, 0]!=0).sum()
            if nonzero_sum == 0:
                v_input[count, 0] = 0 # 这个v好像是论文里面 scale handling部分
            else:
                v_input[count, 0] = np.true_divide(x_input[count, 1:input_size, 0].sum(),nonzero_sum)+1
                x_input[count, :, 0] = x_input[count, :, 0]/v_input[count, 0]
                if train:
                    label[count, :] = label[count, :]/v_input[count, 0]
            count += 1
    # reformat the data, make every batch has all time series
    res=[]
    for i in range(x_input.shape[0]):
        res.append(x_input[i][:,-1][0])
    n1 = Counter(res)[0]

    new_x_input = []
    for i in range(n1):
        for j in range(num_series):
            # print(i+j*n1)
            new_x_input.append(x_input[i+j*n1])
    new_x_input = np.array(new_x_input)

    new_v_input = []
    for i in range(n1):
        for j in range(num_series):
            # print(i+j*n1)
            new_v_input.append(v_input[i+j*n1])
    new_v_input = np.array(new_v_input)

    new_label = []
    for i in range(n1):
        for j in range(num_series):
            # print(i+j*n1)
            new_label.append(label[i+j*n1])
    new_label = np.array(new_label)
    return new_x_input, new_v_input, new_label

In [9]:
train_x_input, train_v_input, train_label = prep_data(train_data, covariates, data_start)
test_x_input, test_v_input, test_label = prep_data(test_data, covariates, data_start, train=False)

100%|██████████| 85/85 [00:00<00:00, 1739.66it/s]


In [19]:
np.save('data/tourism/train_data_tourism', train_x_input)
np.save('data/tourism/train_v_tourism', train_v_input)
np.save('data/tourism/train_label_tourism',train_label)

In [20]:
np.save('data/tourism/test_data_tourism', test_x_input)
np.save('data/tourism/test_v_tourism', test_v_input)
np.save('data/tourism/test_label_tourism',test_label)

In [19]:
final_hier_dict = {}
for k in hier_dict.keys():
    temp_list = hier_dict[k] 
    final_hier_dict['total'+'_'+k] = ['total'+'_'+t for t in temp_list]
temp_list = list(final_hier_dict.keys())
final_hier_dict['total'] = temp_list

In [30]:
np.save('data/tourism/hier_dict_name.npy', final_hier_dict)

In [22]:
final_hier_dict2 = {}
for k in final_hier_dict.keys():
    final_hier_dict2[list(data.columns).index(k)] = [list(data.columns).index(i) for i in final_hier_dict[k]]

In [31]:
pd.DataFrame(list(data.columns)).to_csv('data/tourism/series_names.csv')

In [23]:
final_hier_dict2

{77: [39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50],
 78: [14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26],
 79: [72, 73, 74, 75, 76],
 80: [51,
  52,
  53,
  54,
  55,
  56,
  57,
  58,
  59,
  60,
  61,
  62,
  63,
  64,
  65,
  66,
  67,
  68,
  69,
  70,
  71],
 81: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13],
 82: [27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38],
 83: [0],
 84: [77, 78, 79, 80, 81, 82, 83]}

In [7]:
np.save('data/tourism/hier_dict.npy', final_hier_dict2)

NameError: name 'final_hier_dict2' is not defined

In [8]:
hier_dict = np.load('data/tourism/train_data_tourism.npy',allow_pickle=True)

In [11]:
hier_dict.shape

(2635, 8, 5)

In [12]:
2635/85

31.0